In [1]:
import sys
sys.path.append("../")
import numpy as np
import scripts.prediction_utils as predict_u
import scripts.theodolite_utils as theodo_u
import scripts.theodolite_plot_function as theodo_p
import scripts.groundtruth_utils as theodo_g
import scripts.theodolite_function as theodo_f
import importlib
theodo_u = importlib.reload(theodo_u)
theodo_p = importlib.reload(theodo_p)
theodo_f = importlib.reload(theodo_f)
predict_u = importlib.reload(predict_u)

In [10]:
%matplotlib notebook
theodo_u = importlib.reload(theodo_u)
theodo_p = importlib.reload(theodo_p)
predict_u = importlib.reload(predict_u)
theodo_g = importlib.reload(theodo_g)

## Path of main files
path = [
    # "../data/20220224/",
    # "../data/20220307/",
    # "../data/20220312/",
    # "../data/20220314/",
    # "../data/20220316/",
    # "../data/20220331-1/",
    # "../data/20220331-2/",
    # "../data/20220427-1/",
    # "../data/20220427-2/",
    # "../data/20220505-1/",
    # "../data/20220505-2/",
    # "../data/20220513-1/",
    # "../data/20220513-2/",
    # "../data/20220513-3/",
    # "../data/20220513-4/",
    # "../data/20220513-5/",
    # "../data/20220513-6/",
    # "../data/20220523-1/",
    # "../data/20220523-2/",
    # "../data/20220523-3/",
    # "../data/20220523-4/",
    # "../data/20220525-1/",
    # "../data/20220525-2/",
    "../data/20220622-1/",
    "../data/20220622-2/",
    "../data/20220630-1/",
    "../data/20220630-2/",
    "../data/20220711-1/",
    "../data/20220711-2/",
    "../data/20220715-1/",
    "../data/20220715-2/",
    "../data/20220715-3/",
    "../data/20220715-4/",
    "../data/20220717-1/",
    "../data/20220717-2/",
    "../data/20220717-3/",
    "../data/20220717-4/",
    "../data/20220717-5/",
    "../data/20220910-1/",
    "../data/20220910-2/",
    "../data/20220910-3/",
    "../data/20220910-4/",
    "../data/20220910-5/",
    "../data/20220910-6/",
    "../data/20220910-7/",
    "../data/20220910-8/",
    "../data/20220910-9/",
    "../data/20221103-1/",
    "../data/20221103-2/",
    "../data/20221103-3/",
    "../data/20221109-1/",
    "../data/20221109-2/",
    "../data/20221109-3/",
    "../data/20221110/",
    "../data/20221116-1/",
    "../data/20221123/",
    "../data/20221124/",
    "../data/20221129-1/",
    "../data/20221129-2/",
    "../data/20221129-3/",
    "../data/20221129-4/",
    "../data/20221129-5/",
    "../data/20221205-1/",
    "../data/20221205-2/",
    "../data/20221205-3/"
]

## Parameters for pre-processing pipeline
file = [
    # "/home/maxime/data/ICRA_2023/Vaidis2022_dataset/20220224/20220224_inter_prism.bag",
    # "/home/maxime/data/ICRA_2023/20220307_TS/2022-03-07-19-20-06.bag",
    # "/home/maxime/data/ICRA_2023/20220312_TS/2022-03-12-09-45-12.bag",
    # "/home/maxime/data/ICRA_2023/20220314_TS/2022-03-14-10-47-49.bag",
    # "/home/maxime/data/ICRA_2023/20220316_TS/2022-03-16-19-02-42.bag",
    # "/home/maxime/data/ICRA_2023/20220331_TS/2022-03-31-10-22-52.bag",
    # "/home/maxime/data/ICRA_2023/20220331_TS/2022-03-31-11-20-05.bag",
    # "/home/maxime/data/ICRA_2023/20220427_TS/2022-04-27-12-12-10_filered.bag",
    # "/home/maxime/data/ICRA_2023/20220427_TS_2/2022-04-27-22-33-19_filtered.bag",
    # "/home/maxime/data/ICRA_2023/20220505_TS/cones1_2022-05-05-19-25-54.bag",
    # "/home/maxime/data/ICRA_2023/20220505_TS/empty1_2022-05-05-19-14-33.bag",
    # "/home/maxime/data/ICRA_2023/20220513_TS/2022-05-13-10-30-37_filtered.bag",
    # "/home/maxime/data/ICRA_2023/20220513_TS/2022-05-13-11-01-56.bag",
    # "/home/maxime/data/ICRA_2023/20220513_TS/2022-05-13-11-24-28.bag",
    # "/home/maxime/data/ICRA_2023/20220513_TS/2022-05-13-21-49-12_filtered.bag",
    # "/home/maxime/data/ICRA_2023/20220513_TS/2022-05-13-22-35-30.bag",
    # "/home/maxime/data/ICRA_2023/20220513_TS/2022-05-13-22-51-40.bag",
    # "/home/maxime/data/ICRA_2023/20220523_TS/constrained_2022-05-23-18-56-16.bag",
    # "/home/maxime/data/ICRA_2023/20220523_TS/cones_filtered_2022-05-23-19-31-32.bag",
    # "/home/maxime/data/ICRA_2023/20220523_TS/empty_filtered_2022-05-23-19-18-35.bag",
    # "/home/maxime/data/ICRA_2023/20220523_TS/other_tunnel_2022-05-23-19-39-40.bag",
    # "/home/maxime/data/ICRA_2023/20220525_TS/2022-05-25-11-07-40.bag",
    # "/home/maxime/data/ICRA_2023/20220525_TS/2022-05-25-13-32-32.bag",
    "/home/maxime/data/ICRA_2023/20220622_TS/2022-06-22-17-48-25.bag",
    "/home/maxime/data/ICRA_2023/20220622_TS/2022-06-22-18-11-00.bag",
    "/home/maxime/data/ICRA_2023/20220630_TS/2022-06-30-10-32-21.bag",
    "/home/maxime/data/ICRA_2023/20220630_TS/2022-06-30-10-57-07.bag",
    "/home/maxime/data/ICRA_2023/20220711_TS/2022-07-11-15-36-04.bag",
    "/home/maxime/data/ICRA_2023/20220711_TS/2022-07-11-15-50-00.bag",
    "/home/maxime/data/ICRA_2023/20220715_TS/2022-07-15-11-50-15.bag",
    "/home/maxime/data/ICRA_2023/20220715_TS/2022-07-15-11-53-12.bag",
    "/home/maxime/data/ICRA_2023/20220715_TS/2022-07-15-11-58-58.bag",
    "/home/maxime/data/ICRA_2023/20220715_TS/2022-07-15-14-42-27.bag",
    "/home/maxime/data/ICRA_2023/20220717_TS/constrained_2022-07-17-20-10-31.bag",
    "/home/maxime/data/ICRA_2023/20220717_TS/constrained_2022-07-17-20-11-10.bag",
    "/home/maxime/data/ICRA_2023/20220717_TS/cones_2022-07-17-20-38-29.bag",
    "/home/maxime/data/ICRA_2023/20220717_TS/empty_2022-07-17-21-03-14.bag",
    "/home/maxime/data/ICRA_2023/20220717_TS/calibration_2022-07-17-21-27-28.bag",
    "/home/maxime/data/ICRA_2023/20220910_TS/2022-09-10-12-27-58.bag",
    "/home/maxime/data/ICRA_2023/20220910_TS/2022-09-10-12-29-00.bag",
    "/home/maxime/data/ICRA_2023/20220910_TS/2022-09-10-12-30-49.bag",
    "/home/maxime/data/ICRA_2023/20220910_TS/2022-09-10-12-32-47.bag",
    "/home/maxime/data/ICRA_2023/20220910_TS/2022-09-10-12-34-15.bag",
    "/home/maxime/data/ICRA_2023/20220910_TS/2022-09-10-12-37-00.bag",
    "/home/maxime/data/ICRA_2023/20220910_TS/2022-09-10-13-37-52.bag",
    "/home/maxime/data/ICRA_2023/20220910_TS/2022-09-10-13-49-07.bag",
    "/home/maxime/data/ICRA_2023/20220910_TS/2022-09-10-13-58-19.bag",
    "/home/maxime/data/FR2023/20221103/20221103_1_inter_prism.bag",
    "/home/maxime/data/FR2023/20221103/20221103_2_inter_prism.bag",
    "/home/maxime/data/FR2023/20221103/20221103_3_inter_prism.bag",
    "/home/maxime/data/FR2023/20221109_matin/2022-11-09-11-20-48.bag",
    "/home/maxime/data/FR2023/20221109_matin/2022-11-09-11-58-04.bag",
    "/home/maxime/data/FR2023/20221109_soir/2022-11-09-22-23-59.bag",
    "/home/maxime/data/FR2023/20221110/2022-11-10-10-43-57.bag",
    "/home/maxime/data/FR2023/20221116/2022-11-16-09-47-28.bag",
    "/home/maxime/data/FR2023/20221123/2022-11-23-17-41-30.bag",
    "/home/maxime/data/FR2023/20221124/2022-11-24-18-19-44.bag",
    "/home/maxime/data/FR2023/20221129/2022-11-29-14-34-17.bag",
    "/home/maxime/data/FR2023/20221129/2022-11-29-15-04-19.bag",
    "/home/maxime/data/FR2023/20221129/2022-11-29-15-34-50.bag",
    "/home/maxime/data/FR2023/20221129/2022-11-29-16-09-07.bag",
    "/home/maxime/data/FR2023/20221129/2022-11-29-16-46-19.bag",
    "/home/maxime/data/FR2023/20221205/2022-12-05-15-31-35.bag",
    "/home/maxime/data/FR2023/20221205/2022-12-05-16-26-31.bag",
    "/home/maxime/data/FR2023/20221205/2022-12-05-16-33-34.bag"
]

parameters = [1,2,1,1,3,2]  # 1. Apply filtering or not (Module 1), 2-3-4. Parameters tau_r, tau_a, tau_e (Module 1), 5. Parameter tau_s (Module 2), 6 Parameter tau_l (Module 4).

In [11]:
predict_u = importlib.reload(predict_u)
for path_i, rosbag_i in zip(path, file):
    print(rosbag_i)

    thresold_d = parameters[1]  ## tau_r [m/s]
    thresold_a = parameters[2]  ## tau_a [deg/s]
    thresold_e = parameters[3]  ## tau_e [deg/s]
    limit_time_interval = parameters[4]  ## tau_s [s]

    t1, t2, t3, _, _, _, d1, d2, d3, a1, a2, a3, e1, e2, e3 = theodo_u.read_rosbag_theodolite_without_tf_raw_data_pre_filtered(rosbag_i)
    index_1_f = theodo_u.thresold_raw_data(t1, d1, a1, e1, thresold_d, thresold_a * 3.1415926 / 180,
                                           thresold_e * 3.1415926 / 180, limit_time_interval)
    index_2_f = theodo_u.thresold_raw_data(t2, d2, a2, e2, thresold_d, thresold_a * 3.1415926 / 180,
                                           thresold_e * 3.1415926 / 180, limit_time_interval)
    index_3_f = theodo_u.thresold_raw_data(t3, d3, a3, e3, thresold_d, thresold_a * 3.1415926 / 180,
                                           thresold_e * 3.1415926 / 180, limit_time_interval)
    T1, D1, A1, E1 = predict_u.data_L_Raw_data(t1, d1, a1, e1, index_1_f)
    T2, D2, A2, E2 = predict_u.data_L_Raw_data(t2, d2, a2, e2, index_2_f)
    T3, D3, A3, E3 = predict_u.data_L_Raw_data(t3, d3, a3, e3, index_3_f)
    print(len(T1), len(T2), len(T3))
    theodo_u.save_raw_data_uncertainty(T1, D1, E1, A1, path_i+"uncertainty/raw_data/prism1.csv")
    theodo_u.save_raw_data_uncertainty(T2, D2, E2, A2, path_i+"uncertainty/raw_data/prism2.csv")
    theodo_u.save_raw_data_uncertainty(T3, D3, E3, A3, path_i+"uncertainty/raw_data/prism3.csv")

/home/maxime/data/ICRA_2023/20220622_TS/2022-06-22-17-48-25.bag
Number of data for theodolites: [2321 2283 2246]
Bad measures: 785
2049 2033 2020
Conversion done !
Conversion done !
Conversion done !
/home/maxime/data/ICRA_2023/20220622_TS/2022-06-22-18-11-00.bag
Number of data for theodolites: [1936 2134 2304]
Bad measures: 351
1782 1973 2126
Conversion done !
Conversion done !
Conversion done !
/home/maxime/data/ICRA_2023/20220630_TS/2022-06-30-10-32-21.bag
Number of data for theodolites: [2809 2565 2199]
Bad measures: 1266
1802 1619 1286
Conversion done !
Conversion done !
Conversion done !
/home/maxime/data/ICRA_2023/20220630_TS/2022-06-30-10-57-07.bag
Number of data for theodolites: [3144 2757 3053]
Bad measures: 517
2078 1827 2116
Conversion done !
Conversion done !
Conversion done !
/home/maxime/data/ICRA_2023/20220711_TS/2022-07-11-15-36-04.bag
Number of data for theodolites: [1013  992 1013]
Bad measures: 52
678 682 676
Conversion done !
Conversion done !
Conversion done !
/ho